In [ ]:
#toxic question classification final exam

**Mô tả bài toán:**
Trong thời đại phát triển của công nghệ số, công nghệ cùng với mạng internet đã kết nối con người từ mọi lục địa trên khắp trái đất lại với nhau chỉ trong một tích tăc. Các diễn đàn cùng với đó là các nền tảng mạng xã hội ngày một trở nên quan trọng hơn. Tuy nhiên chúng như những nơi công cộng khác, không gì có thể đảm bảo sự văn minh của những người tham gia được cả. Chính vì thế, để góp phần đảm bảo một môi trường văn minh và đẹp đẽ, Quora đã áp dụng phương pháp dùng học máy để hỗ trợ nhận diện những câu hỏi toxic trong các topic!

Những câu hỏi được đặt ra trên diễn đàn này có đủ các thể loại. Đa phần các câu hỏi được yêu cầu để xử lý trong đề tài đều là những câu hỏi tiếng anh và hoặc là chúng toxic hoặc chỉ là những câu hỏi bình thường. Việc của mô hình bài toán đó là phân loại xem câu hỏi nào là câu hỏi toxic và câu hỏi nào không! 
- Input: Một câu hỏi
- Output: Đánh giá toxic hoặc không toxic

In [ ]:
import numpy as np
import pandas as pd

# natural language tool kit
import nltk
from nltk.corpus import stopwords
import string

#sklearn things
from sklearn.model_selection import train_test_split as df_split
from sklearn.naive_bayes import MultinomialNB

**Miêu tả mô hình:**
mô hình được sử dụng là Multinomial Naive Bayes
Thuật toán Multinomial Naive Bayes là phương pháp học máy dựa trên xác suất thường sử dụng trong xử lý ngôn ngữ - NLP (Natural Language Processing). Thuật toán dựa trên nguyên lý Bayes và dự đoán label của một tập hợp từ. Nó tính xác suất của mỗi label từ những feature và sau đó đưa ra cái label có tỉ lệ cao nhất.  

Nguyên lý Bayes, được phát triển bởi Thomas Bayes, được sử dụng để tính toán tỉ lệ xảy ra của một sự kiện dựa trên những điều kiện của những sự kiện đã biết xảy ra trước đó. Để biểu diễn nguyên lý dưới dạng toán học ta có công thức:

**P(A|B) = P(A) * P(B|A)/P(B)**
Trong đó: 
- P(A), P(B): là xác suất của A và B
- P(B|A): là xác suất của A khi đã biết tỉ lệ xác suất của B

Mô hình này có những ưu nhược điểm sau:
- Ưu điểm:
1. Có thể dễ dàng sử dụng nếu như chỉ cần tính xác suất
2. Có thể sử dụng trên cả những dữ liệu liên tục hay rời rạc.
3. Thuật toán đơn giản và có thể được dùng để dự đoán trong những ứng dụng thơì gian thật.
4. Linh hoạt và dễ dàng làm việc với những ứng dụng thời gian thực.

- Nhược điểm:
1. Độ chính xác của thuật toán thấp hơn những thuật toán khác.
2. Thuật toán không phù hợp với những bài toán hồi quy.Thuật toán Naive Bayes chỉ sử dụng cho việc phân loại dữ liệu dạng xâu ký tự và không thể dự đoán các dữ liệu số

In [ ]:
#get data
path_train = '../input/quora-insincere-questions-classification/train.csv'
path_test = '../input/quora-insincere-questions-classification/test.csv'

raw_df_train = pd.read_csv(path_train)
raw_df_train.head(10)

**Phân tích dữ liệu**
Tệp dữ liệu của chúng ta bao gồm:
1. qid(Question ID): Dạng chuỗi ký tự đánh dấu phân biệt các câu hỏi với nhau và độc nhất. 
2. Question_text: là nội dung của các câu hỏi của Quora sẽ được sử dụng để phân tích và huấn luyện mô hình. Đây là sẽ trở thành các feature của mô hình. 
3. Target: Phân biệt giữa những câu hỏi toxic và các câu hỏi bình thường. Đây là label của mô hình

In [ ]:
raw_df_train.groupby('target').describe()

Ta có thể nhận thấy: 
- Số lượng câu hỏi toxic: 80810
- Số lượng câu hỏi bình thường: 1225312
Số lượng câu hỏi toxic đang bé hơn lượng câu hỏi bình thường. Giả sử trong trường hợp nếu ta sử dụng toàn bộ dataset để huấn luyện mô hình sẽ gây ra tình trạng overfit với các câu hỏi bình thường. Như thế ta sẽ sampling tập câu hỏi không toxic sau đây để đảm bảo điều này không xảy ra.

In [ ]:
#download stop word
nltk.download('stopwords')

Các stop word là các từ không mang quá nhiều ý nghĩa cho nội dung chính của câu và có thể gây confuse cho mô hình, vì thế ta sẽ lược bỏ
ví dụ của stopwords: a, an , the, ...

In [ ]:
#declare a function for pre-processing sentence before feed into the model
def preprocess_text(text):
    #remove punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    #remove stopwords
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    #return a list of clean text words
    return clean_words

**Tiền xử lý dữ liệu:**
Hàm tiền xử lý dữ liệu để đưa một câu về thành dạng các feature:
1. Một câu được đưa vào hàm sẽ bị tách rời rạc các từ trong câu ra.
2. Tập các từ rời rạc sẽ bị lược bỏ đi các từ stop word.
3. Sau đó trả ra một list các từ mang ý nghĩa chính của câu, chính là các feature.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
temp_text = 'I do not know but this is so damn bad, why it is so hard. wish i could quit cause i am so tired'
print('check 1: '+str(preprocess_text(temp_text)))
temp = CountVectorizer(analyzer = preprocess_text).fit_transform([temp_text])
print('check 2: '+str(temp))

Ví dụ: Câu "I do not know but this is so damn bad, why it is so hard. wish i could quit cause i am so tired"
Được đưa vào hàm tiền xử lý sẽ được tách rời và lược bỏ đi các từ stopword. 
Kết quả trả về là một chuỗi như ở check_1

Sau đó ta sẽ sử dụng phương pháp One Hot Coding với các từ trên. ở đây chỉ là thử nghiệm nên có thể thấy các hệ số biểu diễn đều được set lên 1.
Tuy nhiên trong thực tế sau khi huấn luyện thì các feature được xắp xếp trở thành một chuỗi từ điển mà với mỗi một câu đưa vào thì thực chất ta đưa vào mô hình một "chuỗi từ điển" mà các feature trong câu được set lên 1 tại vị trí tương ứng.

In [ ]:
#separate question by it's label
group_non_toxic = raw_df_train[raw_df_train['target']==0]
group_toxic = raw_df_train[raw_df_train['target']==1]

#down sampling non-toxic question
downsampled_non_toxic = group_non_toxic.sample(group_toxic.shape[0]*10)
print('raw group toxic size: '+str(group_toxic.shape[0]))
print('raw downsampled_non_toxic size: '+str(downsampled_non_toxic.shape[0]))

In [ ]:
#mix toxic and non-toxic quest to a new dataset
train_df = pd.concat([group_toxic, downsampled_non_toxic])
print(train_df.shape)

Sau khi downsampling lượng câu hỏi non-toxic, ta gộp 2 tập câu hỏi lại thành 1 tập lớn và trộn đều thành một tập dataset lớn và sử dụng trong quá trình training model

In [ ]:
CV = CountVectorizer(analyzer = preprocess_text)

Khởi tạo một bộ mã hóa One-Hot-Coding lấy từ thư viện của Sklearn

In [ ]:
message_ = CV.fit_transform(train_df['question_text'])

Cho bộ mã hóa xử ký tập các từ có trong bộ dataset sử dụng để huấn luyện.

In [ ]:
x_train, x_test, y_train, y_test  = df_split(message_, train_df['target'],
                                             test_size=0.05, random_state=4, stratify=train_df['target'])

Chia tập dataset ra cho hai mục đích riêng rẽ là huấn luyện và kiểm thử. Tỉ lệ lượng dữ liệu test trên toàn tập dữ liệu là 0.05
Có thêm điều kiện về tỉ lệ cân bằng câu hỏi toxic và non-toxic để có thể cho ra một kết quả trực quan nhất.

In [ ]:
#model classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(x_train,y_train)

Khởi tạo mô hình dự đoán sử dụng thư viện sklearn.
Sau đó ta huấn luyện mô hình với tập dữ liệu train.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(x_test)
print(classification_report(y_test, pred))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print('\nAccuracy: ', accuracy_score(y_test, pred))

Kiếm tra qua mô hình với tập dữ liệu test, ta có thể thấy độ chính xác của mô hình là 0.8619

In [ ]:
text = 'What is the currency in Langkawi?'
print(text)
text = CV.transform([text]).toarray()
print(text)
# print(text)
test_ = classifier.predict(text)
print(test_)

Kiểm thử mô hình với một câu được trích từ file test của cuộc thi.

In [ ]:
raw_df_test = pd.read_csv(path_test)
raw_df_test.head(10)
test_df = CV.transform(raw_df_test['question_text'])
pred_test = classifier.predict(test_df)
print(pred_test)

Chạy dự đoán với dữ liệu từ file test của cuộc thi.

In [ ]:
sumbission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')
sumbission['prediction'] = pred_test

In [ ]:
sumbission.head(20)
sumbission.to_csv("submission.csv", encoding='utf-8', index=False)
# sumbission.to_csv(r"../input/quora-insincere-questions-classification/submission.csv", encoding='utf-8', index=False)

Tận dụng file hướng dẫn sumbit, ta xuất ra các dự đoán của mô hình vào file "submission.csv"